In [1]:
import numpy as np
import pandas as pd

In [2]:
import os, json, math, time

In [3]:
!pip install yelpapi

In [4]:
import sys

In [5]:
sys.path.append("c:/users/om93405/appdata/local/anaconda3/envs/dojo-env/lib/site-packages")

In [6]:
from yelpapi import YelpAPI

In [7]:
from tqdm.notebook import tqdm_notebook

In [8]:
import json
with open('/Users/om93405/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [9]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [10]:
location = 'Seattle, WA 98122'
term = 'pizza'

In [11]:
location.split(',')[0]

'Seattle'

In [12]:
JSON_FILE = "Data/results_in_progress_seattle_pizza.json"
JSON_FILE

'Data/results_in_progress_seattle_pizza.json'

In [13]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_seattle_pizza.json not found. Saving empty list to file.


In [14]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [15]:
results = yelp_api.search_query(location=location,
                                term=term,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
type(results)

dict

In [17]:
len(results)

3

In [18]:
results['total']

858

In [19]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [20]:
results['businesses']

[{'id': 'sx8OeNntGUg3BzQV_L8z6g',
  'alias': 'jackson-street-pizza-lounge-seattle',
  'name': 'Jackson Street Pizza Lounge',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/X9EeTjVjULdmyW9XMowPYA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/jackson-street-pizza-lounge-seattle?adjust_creative=YgmNDOo-e5tb1fZ-0ge60w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=YgmNDOo-e5tb1fZ-0ge60w',
  'review_count': 17,
  'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
   {'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.0,
  'coordinates': {'latitude': 47.59916307243915,
   'longitude': -122.2948217963242},
  'transactions': [],
  'location': {'address1': '2901 S Jackson St',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98144',
   'country': 'US',
   'state': 'WA',
   'display_address': ['2901 S Jackson St', 'Seattle, WA 98144']},
  'phone': '+12064857118',
  'display_phone': '(206) 485-7118',

In [22]:
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

43

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=location,
                                    term=term, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/43 [00:00<?, ?it/s]

In [26]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,sx8OeNntGUg3BzQV_L8z6g,jackson-street-pizza-lounge-seattle,Jackson Street Pizza Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/X9EeTj...,False,https://www.yelp.com/biz/jackson-street-pizza-...,17,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 47.59916307243915, 'longitude': -...",[],"{'address1': '2901 S Jackson St', 'address2': ...",+12064857118,(206) 485-7118,1550.546409,NaN
1,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,143,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}",[pickup],"{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,$$
2,ENxs0vvtkJYxQqaZ3sbEjw,my-friend-dereks-seattle,My Friend Derek's,https://s3-media1.fl.yelpcdn.com/bphoto/MVV6S3...,False,https://www.yelp.com/biz/my-friend-dereks-seat...,34,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 47.651515, 'longitude': -122.343613}",[],"{'address1': '3632 Woodland Park Ave N', 'addr...",,,5581.310444,NaN
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media1.fl.yelpcdn.com/bphoto/UNfR-J...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,370,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]","{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796,$$
4,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,66,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1690.356563,NaN


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
853,Zi4gjjAHv3XcBAs0wSpoJw,qfc-mercer-island-3,QFC,https://s3-media1.fl.yelpcdn.com/bphoto/gbek_5...,False,https://www.yelp.com/biz/qfc-mercer-island-3?a...,61,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 47.58496, 'longitude': -122.23265}",[],"{'address1': '7823 SE 28th St', 'address2': ''...",+12062300745,(206) 230-0745,5672.676413,$$
854,LvUsQR3_dXoPySFQX6zlew,mae-ploy-thai-cuisine-seattle,Mae Ploy Thai Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/p1mbxq...,False,https://www.yelp.com/biz/mae-ploy-thai-cuisine...,133,"[{'alias': 'thai', 'title': 'Thai'}]",3.5,"{'latitude': 47.67577, 'longitude': -122.37667}","[delivery, pickup]","{'address1': '6421 15th Ave NW', 'address2': '...",+12067840899,(206) 784-0899,9247.279220,$$
855,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,68,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 47.6157033283809, 'longitude': -1...",[],"{'address1': '600 Bellevue Way NE', 'address2'...",+14256381000,(425) 638-1000,7123.208902,$$
856,hTvJj509lXVcMptnPBgTcw,auntie-annes-bellevue,Auntie Anne's,https://s3-media4.fl.yelpcdn.com/bphoto/6iE-N9...,False,https://www.yelp.com/biz/auntie-annes-bellevue...,52,"[{'alias': 'pretzels', 'title': 'Pretzels'}]",4.0,"{'latitude': 47.615113, 'longitude': -122.203116}",[],"{'address1': '2040 Bellevue Sq', 'address2': '...",+14256378990,(425) 637-8990,6942.464183,$
857,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media4.fl.yelpcdn.com/bphoto/Dn5kRk...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,299,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"{'latitude': 47.61252119845868, 'longitude': -...","[delivery, pickup]","{'address1': '222 Bellevue Way NE', 'address2'...",+14252141182,(425) 214-1182,7072.982860,$$


In [28]:
final_df.duplicated(subset='id').sum()

0

In [29]:
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)